<a href="https://colab.research.google.com/github/kitt10/semantic_search/blob/main/aq_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Asking Questions Model - Example Usage

##### Download model

In [ ]:
# Downlaod the AQ model from the GDrive
!gdown 1PSaBDLs5z6OF2ej8O_JemzVxNwSaHiic && unzip -u aq.zip


##### Install libs

In [ ]:
!pip install transformers=4.18

##### Initialize the model

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from random import choice


class AQModel:
    """ Asking Questions Model """
    
    def __init__(self):
        self.tokenizer = None
        self.load_tokenizer()
        self.model = None
        self.load_model()
        
    def load_tokenizer(self):
        self.tokenizer = T5Tokenizer('aq/tokenizer/sentencepiece.model')
        print(f'Tokenizer loaded.')

    def load_model(self):
        self.model = T5ForConditionalGeneration.from_pretrained('aq/model', from_tf=True)
        print(f'AQ-Model loaded.')

    def generate(self, context, choose_from=1):
        input_ids = self.tokenizer([context], return_tensors="pt").input_ids
        outputs = self.model.generate(input_ids, 
                                        max_length=128,
                                        num_beams=5,
                                        no_repeat_ngram_size=2, 
                                        num_return_sequences=choose_from, 
                                        early_stopping=True)

        decoded = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        if choose_from > 1:
            print(f'Randomly choosing from: ')
            for d in decoded:
                print(f'{self.parse(d)}')
        
        return self.parse(choice(decoded))
    
    def parse(self, generated):
        try:
            q, a = generated.split('Answer:')
            return q.replace('Question:', '').split('?')[0].strip()+'?', a.strip()
        except:
            print('W: Not parsed.')
            return generated
        

aq = AQModel()

##### Example usage

In [6]:
context = """
    His name is Joe Franklin, and his program ran for 43 years on local 
    television in New York. And he claims that he invented the talk show format.
"""

question, answer = aq.generate(context, choose_from=1)

print(f'Q: {question}')
print(f'A: {answer}')

Q: How long did Joe Franklin's program run on local television?
A: 43 years
